In [1]:
import pandas as pd
import numpy as np
from modules import *
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt
from nltk.util import everygrams
import tensorflow as tf

In [2]:
df = pd.read_csv('comment-labeling.csv', encoding='utf-8-sig')
df1 = pd.read_csv('naver-ratings.csv', encoding='utf-8-sig')

In [3]:
df.columns=['내용','감성']
df1.columns=['내용','감성']

In [4]:
df

,내용,감성
0,인도 기독교 선교 할 수 없는 나라 입니다 . 일반 비자 인도 방문 후 선교 활동 ...,0.0
1,대부분 건전 교단 교류 금지 내려진 단체 귀 신론 김기동 계열 비슷한 닩 제입 니 ...,0.0
2,정말 징글징글 하다 기독교 ... 끔찍해,0.0
3,앞 방역 반해 감염 유발 하는 뭐라고 부를까 ? 코로나 스파이 ? 감염 빨갱이 ? ...,0.0
4,이제 인터콥 정부 기독교 핍박한다고 더 뭉쳐서 기도 장부 비방 히 데모 할것 입니다 .,0.0
...,...,...
968,문 조 도 찐 개 찐 ... 끈끈 ㄱ ㅇ ㅇ ㅊ 관계 본다 .. 뭐 저런 한나라 장...,0.0
969,사기꾼 양성 소 청와대 ! 막 파 양성 소 청와대 !,0.0
970,조국 청와대 사기꾼 뭉친 집단,0.0
971,지금 정부 짓 늘 그런것 새삼 않다 ㆍ 내 로남불 ㆍ 위선 적 공평 평등 ㆍ 위선 ...,0.0


In [5]:
df1

,내용,감성
0,"디자인 배우는 학생 , 외국 디자이너 일군 전통 통해 발전 해가는 문화 산업 부러웠...",1.0
1,폴리스스토리 시리즈 뉴 버릴께 없음 .. 최고 .,1.0
2,.. 연기 진짜 개 쩔구나 .. 지루할거라고 생각 했는데 몰입 해서 봤다 .. 이런...,1.0
3,안개 자욱한 밤하늘 떠 있는 초승달 같은 영화 .,1.0
4,사랑 해본 사람 라면 처음 끝 웃을수 있는 영화,1.0
...,...,...
199994,포켓 몬스터 짜가 ㅡㅡ ;;,0.0
199995,쓰 . 레 . 기,0.0
199996,완전 사이코 영화 . 마지막 영화 질 떨어 트 린다 .,0.0
199997,재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 런가 ㅋㅋ,0.0


In [6]:
df = pd.concat([df,df1])
df

,내용,감성
0,인도 기독교 선교 할 수 없는 나라 입니다 . 일반 비자 인도 방문 후 선교 활동 ...,0.0
1,대부분 건전 교단 교류 금지 내려진 단체 귀 신론 김기동 계열 비슷한 닩 제입 니 ...,0.0
2,정말 징글징글 하다 기독교 ... 끔찍해,0.0
3,앞 방역 반해 감염 유발 하는 뭐라고 부를까 ? 코로나 스파이 ? 감염 빨갱이 ? ...,0.0
4,이제 인터콥 정부 기독교 핍박한다고 더 뭉쳐서 기도 장부 비방 히 데모 할것 입니다 .,0.0
...,...,...
199994,포켓 몬스터 짜가 ㅡㅡ ;;,0.0
199995,쓰 . 레 . 기,0.0
199996,완전 사이코 영화 . 마지막 영화 질 떨어 트 린다 .,0.0
199997,재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 런가 ㅋㅋ,0.0


In [7]:
#원하는 태그 추출 & 불용어 제거
okt = Okt()
comment = okt.pos("댓글 데이터를 입력해주면 형태소 분석이 됩니다.")

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','젛','뇨','껀데','뭘'
                 ,'최고','느리다','마을','최고','항상','포장','재다','말다','목','넘김','만족하다','정말','역시','아주','감사하다','오다','자주','요','늘','넘다','않다','더','병','다','숙성','나다','술','드리다','마트','꼼꼼하다','잇다'
                 ,'복','순도','용','비싸다','가격','많다','강하다','보내다','추천','보고','쏘다','느낌','사다','처음','많이','톡','생각','엄청','날','아니다'
                 ,'믿다', '구입', '사먹다', '한번', '싶다', '콤', '완전', '제품', '재', '해주다', '말', '사람','특별하다','언제나','빨르다','두번째','전','계속','달'
                 ,'추석', '인생', '최애', '명절', '해', '중', '생각나다', '시원하다', '다르다', '알다', '가족', '정도', '기대', '친구', '괜찮다', '부모님', '지인', '아직'
                 ,'만족스럽다', '빼다', '상품', '굿', '못', '도', '기분', '다시', '주다', '즐기다', '맞다', '맛보다', '제일', '집', '입맛', '모르다', '먹기'
                 ,'조심하다', '따다', '차다', '엄마', '예정', '만족', '감', '일반', '파다', '막', '근데', '별로', '적당하다','예전', '가끔', '유명하다', '만', '뚜껑', '매우', '기대하다', '의사', '한잔', '신선하다', '느끼다', '그냥', '아쉽다'
                 ,'오픈', '쎄다', '여자', '호불호', '꼭', '벌써', '버리다', '이렇게', '돼다', '크다', '느껴지다','고급', '종종', '어울리다', '알', '남편', '빨리', '어른', '소문', '아버지', '마시기', '할인', '살다' 
                 ,'풍부하다', '모임', '해드리다', '빠지다', '나서다', '원래', '아빠', '걸리다', '안', '도착', '없이'
                 ,'하루', '딱이다', '상태', '향', '후', '설', '주변', '사서', '때문', '반하다', '연말'
                 ,'해봤다', '강추', '찾다', '자꾸', '살짝', '몇번', '빨', '요즘', '신세계', '사진', '생일'
                 ,'빠르다', '자다', '구매', '마시다', '받다', '보다', '없다', '새다', '숙취', '아스파탐', '대포', '넘어가다','주가'
                 ,'지나다', '무', '머리', '편하다', '아프다', '두다', '인터넷', '택배', '숙성하다', '일주일', '봄', '배달', '만들다', '애용', '냉장고' 
                 ,'여름', '잔', '앞', '며칠', '팔다', '달라지다', '가장', '변하다', '번창', '안전하다', '젤', '배상면', '건강하다', '굿굿'
                 ,'따르다', '이용', '확실하다', '날짜', '시중', '들어가다', '첨가', '유통', '아이스팩', '제조', '넣다', '생', '다양하다','종류','들어가다','전통주','마켓','술술','후기','사보다','부담','여러가지','리뷰','블루'
                 ,'인기','궁금하다','대상','무난','취향','적다','단','평이','이용','터지다','맘'
                 ]
                 +'회랑 회 따뜻하다 나쁘다 튼튼하다 끄다 끝 장인어른 일품 파손 싸다 술맛 비다 착하다 명인 증류 안동소주 안동 뒤끝 박다 신랑 비다 뒤끝 품질 짜다 용이 외국인 지고 지다 높다 어르신 신분 명인 보이다 준비 쇼핑 밤 서울 실향 스컬 낮다 가볍다 섞다 귀엽다 명랑 버전 꼼꼼 볼 대중 패키지 높다 하이 널 타 써다 집들이 실향 원주 긋다 술맛 술집 패키지 결의 증류주 섞다 타 이름 인공 약 높다 감기 대중 인위 짜다 막걸리 대접 볼 관심 타 남다 기회 하이 볼 섞다 향기 도주 집들이 독하다 이름 파티 어떻다 워낙 보기 마치다 가미 다기 수고 맛잇을걱닡아 모으다 햡 담날 대박 물맛 안되다 읺았어 짜다 빠져들다 소비 안나 이색 보리 빨랏 부족하다 화이팅 편 감홍 쇼핑 대다 도자기 생신 술맛 지다 높다 개봉 준비 깊다 볼 외국 아버님 조명 이벤트 용이 얼 눈 신분 보기 작다 내기 불빛 사은 비주 즐겁다 준비 생신 아깝다 주신 싸다 품 아버님 비쥬 조명도 반응 켜다 고맙다 동학 저렴하다 청 서비스 술잔 소주 강쇠주 오메 기술 조선 대비 요리 사용 데우다 조사 쓰다 감사 음식 알콜 경험 청주 강쇠 가성 전주 모주 뜨다 신기하다 화주 좋아하다 주문 예쁘다 이쁘다 얼음 산 토닉 워터 빨대 깨지다 빙 탄복 박스 약하다 국산 오프너 위트 소백산 제공 분위기 아내 크리스마스 마음 마루 베베 솔직하다 초보자 와이프 작성 위트 디자인 굳다 달다 시큼하다 달달 선물 깔끔하다 뒤 비비 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 기한 달라 첫날 첨 주말 걱정 전이 이유 해보다 저번 보지 함 기대다 상 비 베스트 도수 음 존맛 개인'.split()
                 ) #불용어 추가
stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    # n_gram 만들기
    egram = list(everygrams(words, min_len=1, max_len=1))
    egram_token = [' '.join(grams) for grams in egram]

    return egram_token

def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    return words


In [8]:
from sklearn.model_selection import train_test_split

test_percent = 0.2 # 테스트 데이터 비율
# 훈련 데이터와 테스트 데이터 생성
x_train, x_test, y_train, y_test = train_test_split(df['내용'], df['감성'], test_size=test_percent)

In [9]:
from torch import layer_norm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras import Input, Model
from tensorflow.keras import models

def build_model(train_data): # to make rnn model
    train_data = tf.data.Dataset.from_tensor_slices(train_data)
    train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data, value=word_to_index['<PAD>'], padding='post')
    model = Sequential()
    model.add(Input(shape=(1,), dtype="string")) # input one string data (comment)
    max_tokens = 15000 # dictionary size
    max_len = 50 # comment to vectorize size
    vectorize_layer = TextVectorization( # make textvectorization 
      max_tokens=max_tokens,
      output_mode="int",
      output_sequence_length=max_len,
    )
    
    vectorize_layer.adapt(train_data.batch(64))
    model.add(vectorize_layer)
    model.add(layer_norm.Embedding(max_tokens + 1,output_dim= 200))
    model.add(Flatten())
    model.add(Dense(8, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    return model

In [11]:
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

rnn_model =build_model(x_train) # 모델 생성 함수를 통해 모델 생성
rnn_model.compile( # rnn model compile 컴파일 진행
        optimizer=  "adam",  # 최적화 함수
        loss='binary_crossentropy', # 손실 함수
        metrics=['accuracy']
    )

# model training
history = rnn_model.fit(x_train,y_train, #훈련시키기
                   epochs = 5,  batch_size = 128  , validation_data = (x_test, y_test) )

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
model_save_path = "../dir/" # model save path
model_name ="rnn-model" #model save file name
tf.saved_model.save(rnn_model, model_save_path+model_name)